# 1. Importando os dados:

Abaixo, usamos pandas para importar os dados, mas logo transformamos o DataFrame em uma lista, para tentar nos adequar ao máximo ao conteúdo já ministrado.

In [1]:
import pandas as pd

dados = pd.read_json('brasileirao-2019.json')


campeonato = []

for i in dados.columns:
  campeonato.append(list(dados[i]))

# 2. Funções usadas:
Abaixo, criamos as funções que irão ser usadas ao longo do projeto:

In [2]:
def tecnicos(campeonato: list) -> list:
  """retorna a lista de técnicos que participaram do campeonato"""
  tec = sorted(
      list(
          set(
              [
                  tecnico
                  for rodada in campeonato
                  for partida in rodada
                  for tecnico in partida['coach'].values()
              ]
            )
          )
      )
  return tec



def times(campeonato: list) -> list:
  """retorna uma lista com os times que disputaram o campeonato em ordem alfabética"""
  t = sorted(
      list(
          set(
              [
                  time
                  for rodada in campeonato
                  for partida in rodada
                  for time in partida['clubs'].values()
              ]
            )
          )
      )
  return t



def times_por_tecnicos(tecnico: str, campeonato: list) -> dict:
  """Retorna um dicionário, as chaves são times,
  e os valores a quantidade de partidas que o técnico esteve no time"""
  times = [
    partida['clubs'][chave]
    for rodada in campeonato
    for partida in rodada
    for chave in ['home', 'away']
    if partida['coach'][chave] == tecnico
]
  return {time:times.count(time) for time in times}


def chave_tecnico(tecnico: str, partida: dict) -> str:
  """Recebe um tecnico e uma partida, e retorna a chave
  que indica se o tecnico jogou em casa ou fora de casa"""
  if tecnico not in partida['coach'].values():
    return ""
  return [ch for ch in partida['coach'].keys() if partida['coach'][ch] == tecnico][0]



def chave_time(time: str, partida: dict) -> str:
  """Recebe um time e uma partida, e retorna a chave
  que indica se o time jogou em casa ou fora de casa"""
  if time not in partida['clubs'].values():
    return ""
  return [ch for ch in partida['clubs'].keys() if partida['clubs'][ch] == time][0]



def tecnico_de(time: str, partida: dict) -> str:
  """Retorna o ténico do time na partida"""
  return "" if time not in partida['clubs'].values() else partida['coach'][chave_time(time, partida)]



def time_de(tecnico: str, partida: dict) -> str:
    """retorna o time treinado por um tecnico em uma partida"""
    return "" if tecnico not in partida['coach'].values() else partida['clubs'][chave_tecnico(tecnico, partida)]



def vencedor(partida: dict) -> str:
  """recebe uma partida e retorna o time vencedor"""
  if partida['goals']['home'] == partida['goals']['away']:
    return "empate"
  else:
    home = int(partida['goals']['home'])
    away = int(partida['goals']['away'])
    return partida['clubs']['home'] if home > away else partida['clubs']['away']



def pontos(time: str, campeonato: list) -> int:
  """retorna a quantidade de pontos de um time em um campeonato"""
  pontos = 0
  partidas = [
      partida
      for rodada in campeonato
      for partida in rodada
      if time in partida['clubs'].values()
    ]
  for partida in partidas:
    if vencedor(partida) == time:
      pontos += 3
    elif vencedor(partida) == 'empate':
      pontos += 1
  return pontos



def pontos_tecnico(tecnico: str, time: str, campeonato: list) -> int:
    """Retorna a quantidade de pontos que um técnico conseguiu para um dado time em um campeonato"""
    pontos = 0
    for rodada in campeonato:
        for partida in rodada:
            if time in partida['clubs'].values() and tecnico_de(time, partida) == tecnico:
                if vencedor(partida) == time:
                    pontos += 3
                elif vencedor(partida) == 'empate':
                    pontos += 1
    return pontos

# 3. Pontos por Partida. 

Para tentar mensurar a eficiência de um técnico em um time, vamos calcular a razão entre a quantidade de pontos que o time obteve sob o comando do técnico e a quantidade de partidas disputadas pelo time sob o comando do mesmo técnico. 

Podemos ver em quais times um técnico atuou durante um campeonato, e de quantas partidas participou por cada time. Abaixo um exemplo: 

In [3]:
times_por_tecnicos("R. Ceni", campeonato)

{'Fortaleza': 29, 'Cruzeiro': 7}

Abaixo, criamos um dicionário `pontos_por_partida` onde as chaves são os técnicos que participaram do campeonato em questão. Os valores, por sua vez, são dicionários, onde as chaves são times e os valores são a eficiência conforme definimos:

In [4]:
tecs = tecnicos(campeonato)

pontos_por_partida = {tec:
     {
         time:
         pontos_tecnico(tec, time, campeonato)/(times_por_tecnicos(tec, campeonato)[time])
         for time in times_por_tecnicos(tec, campeonato).keys()
     } 
     for tec in tecs}

pontos_por_partida

{'A. Batista': {'Ceará SC': 1.0769230769230769, 'Cruzeiro': 0.0},
 'A. Fucks': {'CSA': 1.0, 'Ceará SC': 0.6666666666666666},
 'A. Lopes dos Santos': {'Palmeiras': 3.0},
 'A. Stival': {'São Paulo': 1.6666666666666667},
 'A. Valentim do Carmo Neto': {'Avaí': 0.8666666666666667, 'Botafogo': 1.0},
 'A. da Silva Braga': {'Flamengo': 1.6666666666666667,
  'Cruzeiro': 1.2142857142857142},
 'B. Lazaroni': {'Botafogo': 1.5},
 'C. A. P. F. Hembert': {'Fortaleza': 0.0},
 'C. dos Santos Oliveira': {'Goiás': 1.4166666666666667},
 'Coelho': {'Corinthians': 1.375},
 'E. Alves Moreira': {'Ceará SC': 1.0454545454545454},
 'E. Hartkopp': {'Chapecoense': 0.75},
 'E. Machado Souto': {'Avaí': 0.4444444444444444},
 'E. Spinassé Camillato': {'Avaí': 0.21428571428571427},
 'E. de Barros': {'Athletico-PR': 2.25},
 'E. de Souza Barroca': {'Botafogo': 1.173913043478261},
 'F. Carille': {'Corinthians': 1.5},
 'F. Diniz Silva': {'Fluminense': 0.8, 'São Paulo': 1.5625},
 'G. Santos Vasconcelo': {'CSA': 0.0},
 'J. M

Entretanto, há muitos técnicos que não tiveram tanta participação no campeonato. Alguns participaram de uma única partida em um único time. Alguns participaram de uma partida em um time e várias em outros. 

O fato é que nem todos os dados que temos são relevantes para nossa análise. Queremos saber se há alguma relação entre o tempo (em rodadas) que um técnico passou em um time e o desempenho do time em um campeonato. Alguns dados além de não serem relevantes, poluem nossa análise, dificultando uma visão mais clara da situação.

Com isso em mente, abaixo vamos considerar, para cada técnico, apenas os times nos quais eles tiveram maior longevidade, ordenaremos a lista pela quantidade de rodadas atuando no time onde foram mais longevos, e descartaremos os técnicos que não chegaram a passar pelo menos 10 rodadas no time onde alcançaram sua maior longevidade no campeonato.

Para isso, primeiro criamos um dicionário `tecnicos_times_partidas` com todos os técnicos que participaram do campeonato, os times em que atuaram e a quantidade de partidas disputadas por cada time:

In [5]:
tecnicos_times_partidas = {tec:times_por_tecnicos(tec, campeonato)
                           for tec in tecs}

tecnicos_times_partidas

{'A. Batista': {'Ceará SC': 13, 'Cruzeiro': 3},
 'A. Fucks': {'CSA': 26, 'Ceará SC': 3},
 'A. Lopes dos Santos': {'Palmeiras': 2},
 'A. Stival': {'São Paulo': 21},
 'A. Valentim do Carmo Neto': {'Avaí': 15, 'Botafogo': 13},
 'A. da Silva Braga': {'Flamengo': 6, 'Cruzeiro': 14},
 'B. Lazaroni': {'Botafogo': 2},
 'C. A. P. F. Hembert': {'Fortaleza': 1},
 'C. dos Santos Oliveira': {'Goiás': 12},
 'Coelho': {'Corinthians': 8},
 'E. Alves Moreira': {'Ceará SC': 22},
 'E. Hartkopp': {'Chapecoense': 8},
 'E. Machado Souto': {'Avaí': 9},
 'E. Spinassé Camillato': {'Avaí': 14},
 'E. de Barros': {'Athletico-PR': 8},
 'E. de Souza Barroca': {'Botafogo': 23},
 'F. Carille': {'Corinthians': 30},
 'F. Diniz Silva': {'Fluminense': 15, 'São Paulo': 16},
 'G. Santos Vasconcelo': {'CSA': 3},
 'J. Mannarino': {'Fortaleza': 7, 'Internacional': 11},
 'J. Pinheiro de Jesus': {'Flamengo': 28},
 'J. Pires de Deus': {'Flamengo': 1},
 'J. Sampaoli Moya': {'Santos': 38},
 'L. Scolari': {'Palmeiras': 16},
 'L. Ve

Abaixo, criamos o dicionário `tecnicos_time_partidas` (não confundir com `tecnicos_times_partidas`) para que nele conste apenas o time onde cada técnico atingiu mais longevidade:

In [6]:
tecnicos_time_partidas = {tec:{   # chave é o técnico, valor é um dicionário 
                time:tecnicos_times_partidas[tec][time]
                    for time in tecnicos_times_partidas[tec].keys()
                    if tecnicos_times_partidas[tec][time] == max(tecnicos_times_partidas[tec].values())
            }  # fim do dicionário interno
            for tec in tecnicos_times_partidas.keys()
        } # fim do dicionário

tecnicos_time_partidas

{'A. Batista': {'Ceará SC': 13},
 'A. Fucks': {'CSA': 26},
 'A. Lopes dos Santos': {'Palmeiras': 2},
 'A. Stival': {'São Paulo': 21},
 'A. Valentim do Carmo Neto': {'Avaí': 15},
 'A. da Silva Braga': {'Cruzeiro': 14},
 'B. Lazaroni': {'Botafogo': 2},
 'C. A. P. F. Hembert': {'Fortaleza': 1},
 'C. dos Santos Oliveira': {'Goiás': 12},
 'Coelho': {'Corinthians': 8},
 'E. Alves Moreira': {'Ceará SC': 22},
 'E. Hartkopp': {'Chapecoense': 8},
 'E. Machado Souto': {'Avaí': 9},
 'E. Spinassé Camillato': {'Avaí': 14},
 'E. de Barros': {'Athletico-PR': 8},
 'E. de Souza Barroca': {'Botafogo': 23},
 'F. Carille': {'Corinthians': 30},
 'F. Diniz Silva': {'São Paulo': 16},
 'G. Santos Vasconcelo': {'CSA': 3},
 'J. Mannarino': {'Internacional': 11},
 'J. Pinheiro de Jesus': {'Flamengo': 28},
 'J. Pires de Deus': {'Flamengo': 1},
 'J. Sampaoli Moya': {'Santos': 38},
 'L. Scolari': {'Palmeiras': 16},
 'L. Venker de Menezes': {'Palmeiras': 20},
 'M. Gomes Valadares': {'Vasco da Gama': 4},
 'M. Longo de

Agora, criamos o dicionário `tecnico_partidas`, que será usado como filtro:

In [7]:
tecnico_partidas = {tec:list(tecnicos_time_partidas[tec].items())[0][1]
                    for tec in tecnicos_time_partidas.keys()}

tecnico_partidas

{'A. Batista': 13,
 'A. Fucks': 26,
 'A. Lopes dos Santos': 2,
 'A. Stival': 21,
 'A. Valentim do Carmo Neto': 15,
 'A. da Silva Braga': 14,
 'B. Lazaroni': 2,
 'C. A. P. F. Hembert': 1,
 'C. dos Santos Oliveira': 12,
 'Coelho': 8,
 'E. Alves Moreira': 22,
 'E. Hartkopp': 8,
 'E. Machado Souto': 9,
 'E. Spinassé Camillato': 14,
 'E. de Barros': 8,
 'E. de Souza Barroca': 23,
 'F. Carille': 30,
 'F. Diniz Silva': 16,
 'G. Santos Vasconcelo': 3,
 'J. Mannarino': 11,
 'J. Pinheiro de Jesus': 28,
 'J. Pires de Deus': 1,
 'J. Sampaoli Moya': 38,
 'L. Scolari': 16,
 'L. Venker de Menezes': 20,
 'M. Gomes Valadares': 4,
 'M. Longo de Araújo': 1,
 'M. Montenegro': 1,
 'M. Ribeiro Cabo': 9,
 'M. Salles': 3,
 'M. de Oliveira': 17,
 'M. dos Santos Gonçalves': 19,
 'N. da Silveira Júnior': 25,
 'O. Hellmann': 24,
 'O. de Oliveira Filho': 6,
 'R. Alves Resende': 1,
 'R. Ceni': 29,
 'R. Colbachini': 3,
 'R. L. Martins Gomes': 1,
 'R. Machado Marques': 38,
 'R. Marques Santana': 25,
 'Renato Gaúcho':

Abaixo, ordenamos o dicionário `tecnico_partidas` pela quantidade de partidas disputadas por cada time

In [8]:
tecnico_partidas = dict(    # transformamos a lista de tuplas abaixo em dicionário
    sorted(                 # ordenamos a lista de tuplas
        tecnico_partidas.items(),   # a lista de tuplas de items chave:valor
        key=lambda item: item[1]    # ordenamos a lista de tuplas pelo valor (que é o segundo elemento da tupla)
    )
)

# copiamos tecnico_partidas removendo os técnicos que participaram de menos de 10 partidas
tecnico_partidas = {tec:tecnico_partidas[tec]
                    for tec in tecnico_partidas.keys()
                   if tecnico_partidas[tec] >= 10}

tecnico_partidas

{'J. Mannarino': 11,
 'C. dos Santos Oliveira': 12,
 'A. Batista': 13,
 'V. Carmo Mancini': 13,
 'A. da Silva Braga': 14,
 'E. Spinassé Camillato': 14,
 'A. Valentim do Carmo Neto': 15,
 'F. Diniz Silva': 16,
 'L. Scolari': 16,
 'M. de Oliveira': 17,
 'M. dos Santos Gonçalves': 19,
 'L. Venker de Menezes': 20,
 'A. Stival': 21,
 'E. Alves Moreira': 22,
 'E. de Souza Barroca': 23,
 'O. Hellmann': 24,
 'N. da Silveira Júnior': 25,
 'R. Marques Santana': 25,
 'A. Fucks': 26,
 'J. Pinheiro de Jesus': 28,
 'R. Ceni': 29,
 'F. Carille': 30,
 'T. Retzalff Nunes': 30,
 'V. Luxemburgo da Silva': 34,
 'J. Sampaoli Moya': 38,
 'R. Machado Marques': 38,
 'Renato Gaúcho': 38}

Finalmente, estamos em condições de filtrar e ordenar nosso dicionário `tecnicos_time_partidas`, e temos em mãos um dicionário com os téncnicos que participaram de pelo menos 10 partidas em um time, junto do time onde atingiram maior longevidade e a quantidade de partidas em que atuaram no respectivo time. 

In [9]:
tecnicos_time_partidas = {tec:tecnicos_time_partidas[tec] for tec in tecnico_partidas}

tecnicos_time_partidas

{'J. Mannarino': {'Internacional': 11},
 'C. dos Santos Oliveira': {'Goiás': 12},
 'A. Batista': {'Ceará SC': 13},
 'V. Carmo Mancini': {'Atlético-MG': 13},
 'A. da Silva Braga': {'Cruzeiro': 14},
 'E. Spinassé Camillato': {'Avaí': 14},
 'A. Valentim do Carmo Neto': {'Avaí': 15},
 'F. Diniz Silva': {'São Paulo': 16},
 'L. Scolari': {'Palmeiras': 16},
 'M. de Oliveira': {'Fluminense': 17},
 'M. dos Santos Gonçalves': {'Chapecoense': 19},
 'L. Venker de Menezes': {'Palmeiras': 20},
 'A. Stival': {'São Paulo': 21},
 'E. Alves Moreira': {'Ceará SC': 22},
 'E. de Souza Barroca': {'Botafogo': 23},
 'O. Hellmann': {'Internacional': 24},
 'N. da Silveira Júnior': {'Goiás': 25},
 'R. Marques Santana': {'Atlético-MG': 25},
 'A. Fucks': {'CSA': 26},
 'J. Pinheiro de Jesus': {'Flamengo': 28},
 'R. Ceni': {'Fortaleza': 29},
 'F. Carille': {'Corinthians': 30},
 'T. Retzalff Nunes': {'Athletico-PR': 30},
 'V. Luxemburgo da Silva': {'Vasco da Gama': 34},
 'J. Sampaoli Moya': {'Santos': 38},
 'R. Macha